In [3]:
import pybamm

model = pybamm.lithium_ion.DFN()
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
experiment = pybamm.Experiment([
        (
            "Discharge at C/5 for 10 hours or until 3.3 V",
            "Charge at 1 A until 4.1 V",
            "Hold at 4.1 V until 10 mA",
        ),
    ]
    * 3
)
sim = pybamm.Simulation(model, experiment=experiment,parameter_values = Para_0,
        solver = pybamm.CasadiSolver(),)

In [4]:
pybamm.set_logging_level("NOTICE")
sim.solve();

2022-10-26 15:20:25.499 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/3 (26.079 ms elapsed) --------------------
2022-10-26 15:20:25.500 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:20:28.875 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:20:31.556 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:20:35.047 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/3 (9.574 s elapsed) --------------------
2022-10-26 15:20:35.048 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:20:37.953 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:20:40.627 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:20:43.843 - [NOTICE] callbacks.on_cycle_star

In [5]:
callback = pybamm.callbacks.LoggingCallback("output.log")
sim.solve(callbacks=callback);

In [6]:
type(callback)

pybamm.callbacks.LoggingCallback

In [7]:
class CustomCallback(pybamm.callbacks.Callback):
    def on_experiment_end(self, logs):
        print(f"We are at the end of the simulation. Logs are {logs}")

In [8]:
rioCall = CustomCallback()
sim.solve(callbacks=rioCall);

2022-10-26 15:22:26.082 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/3 (28.700 us elapsed) --------------------
2022-10-26 15:22:26.083 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:22:29.294 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:22:31.826 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:22:34.562 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/3 (8.480 s elapsed) --------------------
2022-10-26 15:22:34.563 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:22:37.298 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:22:39.693 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:22:42.374 - [NOTICE] callbacks.on_cycle_star

We are at the end of the simulation. Logs are {'stopping conditions': {'voltage': None, 'capacity': None}, 'cycle number': (3, 3), 'elapsed time': pybamm.TimerTime(24.3566414), 'step number': (3, 3), 'step operating conditions': 'Hold at 4.1 V until 10 mA', 'termination': 'event: Current cut-off (negative) [A] [experiment]', 'summary variables': {'Minimum measured discharge capacity [A.h]': 0.4035610768223854, 'Maximum measured discharge capacity [A.h]': 4.4737261006997215, 'Measured capacity [A.h]': 4.070165023877336, 'Minimum voltage [V]': 3.300009999999999, 'Maximum voltage [V]': 4.1, 'Positive electrode capacity [A.h]': 8.7323185203129, 'Change in positive electrode capacity [A.h]': 0.0, 'Loss of active material in positive electrode [%]': 0.0, 'Change in loss of active material in positive electrode [%]': 0.0, 'Loss of lithium inventory [%]': 0.0, 'Change in loss of lithium inventory [%]': 0.0, 'Loss of lithium inventory, including electrolyte [%]': 7.771561172376096e-14, 'Change 

In [14]:
type(rioCall)
print(dir(rioCall))
print(rioCall.on_experiment_infeasible)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'on_cycle_end', 'on_cycle_start', 'on_experiment_end', 'on_experiment_error', 'on_experiment_infeasible', 'on_experiment_start', 'on_step_end', 'on_step_start']
<bound method Callback.on_experiment_infeasible of <__main__.CustomCallback object at 0x000001C957F102E0>>


In [17]:
class RioCallback(pybamm.callbacks.Callback):
    def __init__(self, logfile=None):
        self.logfile = logfile
        self.success  = True
        if logfile is None:
            # Use pybamm's logger, which prints to command line
            self.logger = pybamm.logger
        else:
            # Use a custom logger, this will have its own level so set it to the same
            # level as the pybamm logger (users can override this)
            self.logger = pybamm.get_new_logger(__name__, logfile)
            self.logger.setLevel(pybamm.logger.level)
    
    def on_experiment_error(self, logs):
        self.success  = False
    def on_experiment_infeasible(self, logs):
        self.success  = False
rioCall = RioCallback()
sim.solve(callbacks=rioCall);

2022-10-26 15:38:25.183 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/3 (31.600 us elapsed) --------------------
2022-10-26 15:38:25.184 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:38:28.377 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:38:30.827 - [NOTICE] callbacks.on_step_start(182): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:38:33.483 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/3 (8.301 s elapsed) --------------------
2022-10-26 15:38:33.484 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2022-10-26 15:38:36.145 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2022-10-26 15:38:38.482 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2022-10-26 15:38:41.119 - [NOTICE] callbacks.on_cycle_star

In [18]:
rioCall.success

True